# Building the skeleton of codeOrama from a .sb3 file using Python
Alright. We now know the following:
* How to convert an `.sb3` file to a `.json` file
* How to parse and show its contents
* How to modify a `.json` file
* How to save the changes and create a a new `.sb3` file with the applied changes.

Now that we know the basics, let's focus on how we can utilize the JSON file for codeOrama and eCodeOrama. In codeOrama, each solumn represents a sprite and each row represents an event. Each cell contains scripts of the corresponding sprite that are called on the corresponding event. I think a good stepping stone would be to write a little script that builds the **skeleton** of codeOrama, namely the top row of sprites and first column of events.

For this notebook, I'm going to be using the `events.sb3` file, which contains a Green Flag event, a Broadcast event and a KeyPress event.

## Converting to JSON
This is a standard process that we have already seen a couple of times(converting to zip and unzipping).

In [1]:
!rm -rf ./events/
!cp ../sb3_files/events.sb3 ./events.zip
!unzip ./events.zip -d ./events/
!rm -rf ./events.zip

Archive:  ./events.zip
  inflating: ./events/project.json   
  inflating: ./events/83a9787d4cb6f3b7632b4ddfebf74367.wav  
  inflating: ./events/83c36d806dc92327b9e7049a565c6bff.wav  
  inflating: ./events/b15adefc3c12f758b6dc6a045362532f.wav  
  inflating: ./events/511430b7fa3da45b6ecd429d9005816d.svg  
  inflating: ./events/bcf454acf82e4504149f7ffe07081dbc.svg  
  inflating: ./events/0fb9be3e8397c983338cb71dc84d0b25.svg  
  inflating: ./events/35cd78a8a71546a16c530d0b2d7d5a7f.svg  
  inflating: ./events/d5a72e1eb23a91df4b53c0b16493d1e6.svg  


## Isolating sprites
The first thing that we need to do is isolate the sprites/objects. The following script prints the number of sprites found inside of a JSON file as well as their names.

In [2]:
import json

# Load the JSON file
project_json = None
with open('./events/project.json', 'r', encoding='utf-8') as f:
    project_json = json.load(f)

# Get objects
object_list = project_json["targets"]

# Get object info
object_count = len(object_list)
object_names = []
for object in object_list:
    object_names.append(object["name"])
    
print(f"This Scratch project consists of {object_count} objects. Their names are: ")
for name in object_names:
    print(f"{name}")

This Scratch project consists of 3 objects. Their names are: 
Stage
Cat
Dog1


## Isolating events
Secondly, we want to get all the events of the project. In order to do that, we will iterate over all blocks and save those that have no parent blocks. In the future, we are probably going to need more information than an event's Opcode, but since we only worry about the skeleton of codeOrama, let's keep it simple for now.

In [3]:
import json

# Load the JSON file
project_json = None
with open('./events/project.json', 'r', encoding='utf-8') as f:
    project_json = json.load(f)

# Get objects
object_list = project_json["targets"]

# Event info
event_count = 0
event_names = []

# Check all blocks of all objects
for object in object_list:
    blocks = object["blocks"]

    # Only keep blocks that have no parent block
    for block_id, block_data in blocks.items():
        if block_data["parent"] == None:
            event_count += 1
            event_names.append(block_data["opcode"])

print(f"This Scratch project consists of {event_count} events. Their names are: ")
for name in event_names:
    print(f"{name}")

This Scratch project consists of 7 events. Their names are: 
event_whenflagclicked
event_whenbroadcastreceived
event_whenkeypressed
event_whenthisspriteclicked
event_whenbackdropswitchesto
event_whengreaterthan
event_whengreaterthan


## Building the skeleton
Now that we have access to both the objects and the events, we can implement a final script that builds the skeleton of codeOrama by essentially placing objects on the first row and events on the first column. Python offers a variety of libraries that help build Excel sheets. I'm going to be using `pandas`. Before implementing the script, we need to install the module as well as some dependencies locally.

In [4]:
%pip install pandas
%pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import json             # JSON parsing
import pandas as pd     # Building the Excel sheet

# Load the JSON file
project_json = None
with open('./events/project.json', 'r', encoding='utf-8') as f:
    project_json = json.load(f)

# Get objects
object_list = project_json["targets"]

# Object info
object_count = len(object_list)
object_names = []

# Event info
event_count = 0
event_names = []

# Get object and event info
for object in object_list:
    # Keep object names
    object_names.append(object["name"])

    # Keep blocks that have no parent block
    blocks = object["blocks"]
    for block_id, block_data in blocks.items():
        if block_data["parent"] == None:
            event_count += 1
            event_names.append(block_data["opcode"])
    
# Why not print some statistics
print(f"This Scratch project consists of {object_count} objects. Their names are: ")
for name in object_names:
    print(f"{name}")
print()
print(f"This Scratch project consists of {event_count} events. Their names are: ")
for name in event_names:
    print(f"{name}")
print()

# Build the Excel sheet
print("Building skeleton...")
excel_name = "./codeOrama_skeleton.xlsx"
df = pd.DataFrame(index=event_names, columns=object_names)
df.to_excel(excel_name, index=True)
print(f"Skeleton saved to {excel_name} successfully!")

This Scratch project consists of 3 objects. Their names are: 
Stage
Cat
Dog1

This Scratch project consists of 7 events. Their names are: 
event_whenflagclicked
event_whenbroadcastreceived
event_whenkeypressed
event_whenthisspriteclicked
event_whenbackdropswitchesto
event_whengreaterthan
event_whengreaterthan

Building skeleton...
Skeleton saved to ./codeOrama_skeleton.xlsx successfully!
